<a href="https://colab.research.google.com/github/karim-mammadov/NLP_MyProjects/blob/main/Sentiment_Analysis_Dataset_Binary_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download dineshpiyasamara/sentiment-analysis-dataset

In [ ]:
import zipfile

In [ ]:
with zipfile.ZipFile('/content/sentiment-analysis-dataset.zip','r') as zip_ref:
  zip_ref.extractall('/content')

In [ ]:
import pandas as pd
df = pd.read_csv('/content/sentiment_analysis.csv')

In [ ]:
df.head(5)

In [ ]:
print(df['tweet'].head(5))

In [ ]:
import matplotlib.pyplot as plt

df['text_length'] = df['tweet'].apply(lambda x: len(str(x)))

plt.figure(figsize=(10,5))
plt.hist(df['text_length'], bins=50, color='skyblue')
plt.xlabel('Mətn uzunluğu')
plt.ylabel('Sətir sayı')
plt.title('Mətn uzunluqlarının paylanması')
plt.show();

In [ ]:
from wordcloud import WordCloud

all_text = " ".join(df['tweet'].astype(str))

wordcloud = WordCloud(width=800, height=400, background_color='white').generate(all_text)

plt.figure(figsize=(15,7))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show();

In [ ]:
short_texts = df[df['text_length'] < 30]
print("Çox qısa mətnlər:\n", short_texts['tweet'].head(10))

long_texts = df[df['text_length'] > 300]
print("Çox uzun mətnlər:\n", long_texts['tweet'].head(10))

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

In [ ]:
def clean_text(text):
  text = text.lower()
  text = re.sub(r'http\S+', '', text)
  text = re.sub(r'@\S+', '', text)
  text = re.sub(r'[^a-z\s]', '', text)
  text = ' '.join([word for word in text.split() if word not in stop_words])
  return text
df['cleaned_text'] = df['tweet'].apply(clean_text)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df['cleaned_text'], df['label'], test_size=0.2, random_state=42)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=5000)

X_train_vec = tfidf_vectorizer.fit_transform(X_train)
X_test_vec = tfidf_vectorizer.transform(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

model = LogisticRegression()

model.fit(X_train_vec, y_train)

y_pred = model.predict(X_test_vec)

print(classification_report(y_test, y_pred))

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm = confusion_matrix(y_test, y_pred)

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Positive', 'Negative'])
disp.plot(cmap=plt.cm.Blues)
plt.show();